In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pulp import *
from copy import deepcopy
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser
from src.preprocessing.preprocessor import Preprocessor
from src.solvers.solution import Solution
from src.solvers.pulp_solver import PuLPSolver

from tqdm import tqdm
from typing import Dict

/tmp/ipykernel_428354/804485343.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [170]:
parser = Parser("data/testfiles/test3.txt")

In [171]:

info = parser.read()

28it [00:00, 137840.98it/s]


In [172]:
dataset : Dict[int, pd.DataFrame ]= info['data']
p, K, M, N = info['p'], info['K'], info['M'], info['N']

In [173]:
preprocessor = Preprocessor(
    K, 
    M,
    N,
    p,
    dataset
)

data_1 = preprocessor.remove_trivial_values()
data_2 = preprocessor.remove_ip_dominated(dataset)
data_3 = preprocessor.remove_lp_dominated(data_2)

100%|██████████| 4/4 [00:00<00:00, 1786.90it/s]


In [174]:
def is_integer_solution(solution : LpProblem):
    for var in solution.variables():
        v = var.value()
        if int(v) != v:
            return False
        
    return True

In [175]:
s = PuLPSolver(
    K,
    M,
    N,
    p,
    data_3
)

In [179]:
best_solution = Solution(
    K,
    M,
    N,
    p
)
best_rate = 0

def BB(
    solver : PuLPSolver, 
    n0, 
    remaining_power_budget, 
    data : Dict[int, pd.DataFrame]
):
    global best_solution
    global best_rate
    
    solver.solve()

    if n0 == N-1:
        data_rate =solver.solution.objective.value()
        if data_rate > best_rate:
            best_rate = data_rate

        return
    
    if solver.solution.status != 1:
        return
    
    data_rate =solver.solution.objective.value()
    if is_integer_solution(solver.solution):
        if data_rate > best_rate:
            best_rate = data_rate

        return
    
    if data_rate < best_rate + 1:
        return

    for idx, row in data[n0].iterrows():


        k = row['k']
        m = row['m']
        p_kmn = row['p_k,m,n']

        x_kmn = solver.get_variable(k,m,n0)
        solver.solution.addConstraint(x_kmn == 1)

        BB(
            (solver),
            n0+1,
            remaining_power_budget,
            data_3
        )

        solver.solution.constraints.popitem(-1)



In [180]:
solver = PuLPSolver(
    K,
    M,
    N, 
    p,
    data_3
)
BB(
    solver,
    0,
    p,
    data_3
)

In [181]:
best_rate

350.0

In [153]:
solver.solution.status

1

In [115]:
solver = PuLPSolver(
    K,
    M,
    N, 
    p,
    data_3
)

In [116]:
solver.solve()

In [117]:
k =1
m = 0
n0 = 0

In [118]:
x_kmn = solver.get_variable(k,m,n0)

In [122]:
solver.solution.constraints

OrderedDict([('power_constraint',
              7.0*x_0,0,1 + 2.0*x_0,0,3 + 27.0*x_0,1,0 + 46.0*x_0,1,1 + 1.0*x_1,0,0 + 2.0*x_1,0,1 + 9.0*x_1,1,3 + 1.0*x_2,0,2 + 25.0*x_2,1,2 + -100.0 <= 0),
             ('_C1', 1*x_0,1,0 + 1*x_1,0,0 + -1 = 0),
             ('_C2', 1*x_0,0,1 + 1*x_0,1,1 + 1*x_1,0,1 + -1 = 0),
             ('_C3', 1*x_2,0,2 + 1*x_2,1,2 + -1 = 0),
             ('_C4', 1*x_0,0,3 + 1*x_1,1,3 + -1 = 0)])

In [123]:
solver.solution.constraints.popitem(-1)

('_C4', 1*x_0,0,3 + 1*x_1,1,3 + -1 = 0)

In [68]:
BB(None, 0, [],p, data_3)

1.0 0.0 0
1.0 0.0 1


PulpSolverError: Pulp: Error while executing /home/pedro/anaconda3/envs/user-scheduling-in-5g/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc

In [39]:
best_rate

153.0

In [46]:
best_solution.variablesDict()

AttributeError: 'Solution' object has no attribute 'variablesDict'